## Imports

In [1]:
from constants import FLUXES_FOLDER, SLM_FOLDER
from data_utils import load_numpy_data, flatten_data, shuffle_arrays
from plot_utils import plot_map, plot_model_history
from modeling_utils import create_linear_architecture_for_amplitude_reconstruction, \
                           compile_linear_model_for_amplitude_reconstruction, \
                           train_linear_model_for_amplitude_reconstruction


2023-11-23 09:57:03.093376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-23 09:57:03.957282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load data

In [2]:
fluxes_filename = "all_fluxes.npy"
amplitudes_filename = "complexsine_pupamp.npy"

fluxes_path = f"{FLUXES_FOLDER}/{fluxes_filename}"
amplitudes_path = f"{SLM_FOLDER}/{amplitudes_filename}"


In [13]:
n_points = 10000

fluxes_array = load_numpy_data(fluxes_path, 
                               n_points)

amplitudes_array = load_numpy_data(amplitudes_path,
                                   n_points)

fluxes_array, amplitudes_array = shuffle_arrays([fluxes_array, amplitudes_array])
train_fluxes_array = fluxes_array[0:9000]
val_fluxes_array = fluxes_array[9000:]

train_amplitudes_array = amplitudes_array[0:9000]
val_amplitudes_array = amplitudes_array[9000:]

## Data visualisation

Firstly let's have a look at an amplitude map

In [14]:
plot_map(amplitudes_array[8])

Now let's see how an output flux looks like

In [15]:
plot_map(fluxes_array[8])

Now let's flatten our fluxes to be the input of a linear neural network

In [16]:
flattened_train_fluxes_array = flatten_data(train_fluxes_array)
flattened_val_fluxes_array = flatten_data(val_fluxes_array)

## Model training

Set the configuration

In [17]:
from tensorflow import keras

input_shape = flattened_train_fluxes_array[0].shape
output_shape = amplitudes_array[0].shape
hidden_layer_sizes = [1024, 256, 1024]
regularizer = keras.regularizers.L1L2(l1=0.001,l2=0.1)
hidden_activation = 'relu'
output_activation = 'linear'
use_batch_normalization = True
model_name = "AmplitudeReconstructor1"

Create the model

In [18]:
model = create_linear_architecture_for_amplitude_reconstruction(
    input_shape,
    output_shape,
    hidden_layer_sizes,
    regularizer,
    hidden_activation,
    output_activation,
    use_batch_normalization,
    model_name
)

2023-11-23 09:58:42.135998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 09:58:42.259662: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 09:58:42.259924: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Compile the model

In [19]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

loss_function = tf.keras.losses.MeanSquaredError()
learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
metric = tf.keras.metrics.MeanSquaredError()

compile_linear_model_for_amplitude_reconstruction(
    model,
    loss_function,
    optimizer,
    metric
)

Train the model

In [20]:
epochs = 750
batch_size = 64
reduce_lr = ReduceLROnPlateau('mean_squared_error', factor=0.1, patience=100, verbose=1)
early_stop = EarlyStopping('mean_squared_error', patience=150, verbose=1)
callbacks = [reduce_lr, early_stop]

history = train_linear_model_for_amplitude_reconstruction(
    model,
    flattened_train_fluxes_array,
    train_amplitudes_array,
    batch_size,
    epochs,
    flattened_val_fluxes_array,
    val_amplitudes_array,
    callbacks
)

Epoch 1/750


2023-11-23 09:58:46.414760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-23 09:58:46.574480: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e787000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-23 09:58:46.574510: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-11-23 09:58:46.603432: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-23 09:58:46.846441: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-11-23 09:58:47.028596: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime o

141/141 [==============================] - 5s 10ms/step - loss: 22.0696 - mean_squared_error: 0.2890 - val_loss: 2.1477 - val_mean_squared_error: 0.1085 - lr: 0.0100
Epoch 2/750
141/141 [==============================] - 1s 8ms/step - loss: 1.8120 - mean_squared_error: 0.0386 - val_loss: 1.7090 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 3/750
141/141 [==============================] - 1s 7ms/step - loss: 1.7466 - mean_squared_error: 0.0294 - val_loss: 1.8200 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 4/750
141/141 [==============================] - 1s 8ms/step - loss: 1.8833 - mean_squared_error: 0.0290 - val_loss: 1.9393 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 5/750
141/141 [==============================] - 1s 9ms/step - loss: 1.9819 - mean_squared_error: 0.0290 - val_loss: 2.0235 - val_mean_squared_error: 0.0298 - lr: 0.0100
Epoch 6/750
141/141 [==============================] - 1s 9ms/step - loss: 2.0509 - mean_squared_error: 0.0290 - val_loss: 2.0836 

141/141 [==============================] - 1s 8ms/step - loss: 2.3701 - mean_squared_error: 0.0290 - val_loss: 2.3427 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 48/750
141/141 [==============================] - 1s 9ms/step - loss: 2.3698 - mean_squared_error: 0.0290 - val_loss: 2.2932 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 49/750
141/141 [==============================] - 1s 9ms/step - loss: 2.3697 - mean_squared_error: 0.0290 - val_loss: 2.3822 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 50/750
141/141 [==============================] - 1s 9ms/step - loss: 2.3703 - mean_squared_error: 0.0290 - val_loss: 2.4554 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 51/750
141/141 [==============================] - 1s 9ms/step - loss: 2.3708 - mean_squared_error: 0.0290 - val_loss: 2.3737 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 52/750
141/141 [==============================] - 1s 9ms/step - loss: 2.3707 - mean_squared_error: 0.0290 - val_loss: 2.28

141/141 [==============================] - 1s 8ms/step - loss: 2.3723 - mean_squared_error: 0.0290 - val_loss: 2.3654 - val_mean_squared_error: 0.0293 - lr: 0.0100
Epoch 94/750
141/141 [==============================] - 1s 8ms/step - loss: 2.3726 - mean_squared_error: 0.0290 - val_loss: 2.4411 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 95/750
141/141 [==============================] - 1s 8ms/step - loss: 2.3729 - mean_squared_error: 0.0290 - val_loss: 2.3868 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 96/750
141/141 [==============================] - 1s 8ms/step - loss: 2.3727 - mean_squared_error: 0.0290 - val_loss: 2.3206 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 97/750
141/141 [==============================] - 1s 8ms/step - loss: 2.3724 - mean_squared_error: 0.0290 - val_loss: 2.3597 - val_mean_squared_error: 0.0292 - lr: 0.0100
Epoch 98/750
141/141 [==============================] - 1s 8ms/step - loss: 2.3725 - mean_squared_error: 0.0290 - val_loss: 2.42

...

141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 540/750
141/141 [==============================] - 1s 7ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 541/750
141/141 [==============================] - 1s 7ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 542/750
141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 543/750
141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 544/750
141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error

141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 585/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 586/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 587/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 588/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0292 - val_mean_squared_error: 0.0291 - lr: 1.0000e-07
Epoch 589/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error

141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 629/750
141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 630/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 631/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 632/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 633/750
141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error

141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 674/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 675/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 676/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 677/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-08
Epoch 678/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error

141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-09
Epoch 718/750
141/141 [==============================] - 1s 8ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-09
Epoch 719/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-09
Epoch 720/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-09
Epoch 721/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.0291 - val_mean_squared_error: 0.0291 - lr: 1.0000e-09
Epoch 722/750
141/141 [==============================] - 1s 9ms/step - loss: 0.0289 - mean_squared_error

Inspect the training process

In [21]:
plot_model_history(history)

NameError: name 'plot_model_history' is not defined

In [25]:
n = 6
plot_map(amplitudes_array[n])
im = model.predict(fluxes_array[n].reshape(1, 1320))
plot_map(im[0])

1/1 [==============================] - 0s 16ms/step


Another

In [ ]:
input_shape = fluxes_array[0].shape
model = models.Sequential()

model.add(layers.InputLayer(input_shape=fluxes_array[0].shape))
# Adding a fully connected layer with 128 units and 'relu' activation function
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
# Adding the output layer with 10 units (for example, for 10 classes) and 'softmax' activation function
model.add(layers.Dense(96 * 96, activation='linear'))
model.add(layers.Reshape((96,96)))
# Printing the summary of the model architecture
model.summary()


# Compilation hyperparameters
loss_function = tf.keras.losses.MeanSquaredError()
learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
metric = tf.keras.metrics.MeanSquaredError()

model.compile(loss=loss_function,optimizer=optimizer,metrics=[metric])
history = model.fit(fluxes_array, amplitudes_array, epochs=100, batch_size=16, validation_split=0.2)

In [ ]:
n=1102
plot_map(amplitudes_array[n])
im = model.predict(fluxes_array[n].reshape(1, 1320))
plot_map(im[0])

In [ ]:
n=8002
plot_map(amplitudes_array[n])
im = model.predict(fluxes_array[n].reshape(1, 1320))
plot_map(im[0])

In [ ]:
plot_model_history(history)

Another

In [ ]:
fluxes_array = load_numpy_data(fluxes_path, 
                               n_points)

amplitudes_array = load_numpy_data(amplitudes_path,
                                   n_points)

In [ ]:
model = models.Sequential()
input_shape = fluxes_array[0].shape + (1, )
# Adding a convolutional layer with 32 filters, a 3x3 kernel, and 'relu' activation function
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape))

# Adding another convolutional layer with 64 filters, a 3x3 kernel, and 'relu' activation function
model.add(Conv2D(64, (3, 3), activation='relu'))

# Adding another max-pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the 3D output to 1D tensor for a fully connected layer
model.add(Flatten())

# Adding a fully connected layer with 128 units and 'relu' activation function
model.add(Dense(128, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))

# Adding the output layer with 10 units (for example, for 10 classes) and 'softmax' activation function
model.add(layers.Dense(96 * 96, activation='linear'))
model.add(layers.Reshape((96,96)))
# Printing the summary of the model architecture
model.summary()


# Compilation hyperparameters
loss_function = tf.keras.losses.MeanSquaredError()
learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
metric = tf.keras.metrics.MeanSquaredError()

model.compile(loss=loss_function,optimizer=optimizer,metrics=[metric])
history = model.fit(fluxes_array, amplitudes_array, epochs=100, batch_size=16, validation_split=0.2)

In [ ]:
plot_model_history(history)

In [ ]:
n=1000
plot_map(amplitudes_array[n])
im = model.predict(fluxes_array[n].reshape(1, 1320))
plot_map(im[0])